In [ ]:
import tensorflow as tf

import tensorflow_models as tfm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D,Dense,Input
from tensorflow.keras.utils import pad_sequences
import pandas as pd

In [ ]:
tfm.nlp.layers.TransformerEncoderBlock

official.nlp.modeling.layers.transformer_encoder_block.TransformerEncoderBlock

In [ ]:
(X_train,y_train),(X_test,y_test) = tf.keras.datasets.reuters.load_data(num_words=10000)
y_train =  tf.convert_to_tensor(pd.get_dummies(y_train))
y_test = tf.convert_to_tensor(pd.get_dummies(y_test))

In [ ]:
class PositionEncoding(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def make_values(self, n, d):
        # intialize with the denominator inside of sine values
        pos_enc = 1 / 10000**(2 * tf.range(tf.cast(tf.math.ceil(d/2), tf.float32) / tf.cast(d, tf.float32)))
        
        # multiply by position (numerator)
        pos_enc = pos_enc * tf.cast(tf.repeat(tf.expand_dims(tf.range(n), axis=-1), tf.cast(tf.math.ceil(d/2), tf.int32), axis=-1), tf.float32)
        
        return pos_enc
    def call(self, inputs):
        input_shape = tf.shape(inputs)   # batch, sequence, features
        # computes the inside of sine values
        pos_enc = self.make_values(input_shape[1], input_shape[2])
        
        # stack sine and cosine values
        pos_enc = tf.stack((tf.sin(pos_enc), tf.cos(pos_enc)), axis=-1)
        
        # reshape them to get sine in even and cosine in odd dimensions
        pos_enc = tf.reshape(pos_enc, (input_shape[1], -1))
        
        # repeat for all sample in batch
        pos_enc = tf.repeat(tf.expand_dims(pos_enc, axis=0), input_shape[0], axis=0)
        
        # add to inputs
        return inputs + pos_enc[..., :input_shape[2]]


In [ ]:

maxlen = 200

X_train = tf.convert_to_tensor(pad_sequences(X_train,maxlen=maxlen))
X_test = tf.convert_to_tensor(pad_sequences(X_test,maxlen=maxlen))
print(X_train)

tf.Tensor(
[[  0   0   0 ...  15  17  12]
 [  0   0   0 ... 505  17  12]
 [  0   0   0 ...  11  17  12]
 ...
 [  0   0   0 ... 407  17  12]
 [  0   0   0 ... 364  17  12]
 [  0   0   0 ... 113  17  12]], shape=(8982, 200), dtype=int32)


In [ ]:
model = Sequential()
model.add(Input(shape=(maxlen,)))
model.add(Embedding(maxlen,128,mask_zero=True))
model.add(PositionEncoding())
for i in range(4):
  model.add(tfm.nlp.layers.TransformerEncoderBlock(8,512,"relu"))
model.add(GlobalAveragePooling1D())
model.add(Dense(46,activation="softmax"))

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 200, 128)          25600     
                                                                 
 position_encoding_3 (Positi  (None, 200, 128)         0         
 onEncoding)                                                     
                                                                 
 transformer_encoder_block_1  (None, 200, 128)         198272    
 2 (TransformerEncoderBlock)                                     
                                                                 
 transformer_encoder_block_1  (None, 200, 128)         198272    
 3 (TransformerEncoderBlock)                                     
                                                                 
 transformer_encoder_block_1  (None, 200, 128)         198272    
 4 (TransformerEncoderBlock)                          

In [ ]:
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10)

Epoch 1/10
281/281 [==============================] - 43s 113ms/step - loss: 2.1946 - accuracy: 0.4379 - val_loss: 1.8967 - val_accuracy: 0.5303
Epoch 2/10
281/281 [==============================] - 23s 82ms/step - loss: 1.8492 - accuracy: 0.5212 - val_loss: 1.8597 - val_accuracy: 0.5142
Epoch 3/10
281/281 [==============================] - 19s 67ms/step - loss: 1.7881 - accuracy: 0.5407 - val_loss: 1.7775 - val_accuracy: 0.5450
Epoch 4/10
281/281 [==============================] - 16s 58ms/step - loss: 1.7476 - accuracy: 0.5538 - val_loss: 1.8043 - val_accuracy: 0.5730
Epoch 5/10
281/281 [==============================] - 17s 60ms/step - loss: 1.6966 - accuracy: 0.5733 - val_loss: 1.7423 - val_accuracy: 0.5677
Epoch 6/10
281/281 [==============================] - 15s 54ms/step - loss: 1.7003 - accuracy: 0.5736 - val_loss: 1.9432 - val_accuracy: 0.5530
Epoch 7/10
281/281 [==============================] - 15s 53ms/step - loss: 1.6798 - accuracy: 0.5715 - val_loss: 1.6832 - val_accuracy